# cleaning data
- to use conveniently
- need to install holidays
    - `pip install holidays` (https://github.com/dr-prodigy/python-holidays)

In [1]:
import datetime as dt
import holidays
from taxi_pakage import haversine_np

In [2]:
taxi = pd.read_csv('train.csv')

In [4]:
taxi['dist'] = haversine_np(taxi['pickup_longitude'], taxi['pickup_latitude'], taxi['dropoff_longitude'], taxi['dropoff_latitude'])

In [5]:
datezero = dt.datetime(2016, 1, 1, 0, 0, 1) # 기준
def strptime(x):
    return dt.datetime.strptime(x, "%Y-%m-%d %H:%M:%S")
def date_to_zero(x):
    return int((x-datezero).days)
def time_to_zero(x):
    return int((x-datezero).seconds)
def week_num(x):
    return int(x.weekday())

In [6]:
us_holidays = holidays.US(state='NY', years=2016)
def holiday(x):
    if x in us_holidays: 
        return 1
    else: 
        x = x.weekday()
    if x > 4: 
        return 1
    else: 
        return 0
    
def holiday_Fri(x): #with Friday
    if x in us_holidays: 
        return 1
    else: 
        x = x.weekday()
    if x > 3:
        return 1
    else: 
        return 0

In [7]:
taxi['id'] = taxi['id'].apply(lambda x: x[2:])
taxi['store_and_fwd_flag'] = taxi['store_and_fwd_flag'].apply(lambda x: 0 if x == 'N' else 1)

taxi['pickup_datetime'] = taxi['pickup_datetime'].apply(strptime)
taxi['dropoff_datetime'] = taxi['dropoff_datetime'].apply(strptime)
taxi['pick_date'] = taxi['pickup_datetime'].apply(date_to_zero)
taxi['pick_time'] = taxi['pickup_datetime'].apply(time_to_zero)
taxi['drop_time'] = taxi['dropoff_datetime'].apply(time_to_zero)

taxi['holiday'] = taxi['pickup_datetime'].apply(holiday)
taxi['holiday_Fri'] = taxi['pickup_datetime'].apply(holiday_Fri)
taxi['weekday'] = taxi['pickup_datetime'].apply(week_num)


# taxi['velo'] = taxi['dist']/taxi['trip_duration']*3600 #km/h

In [8]:
temp = taxi['trip_duration'] # for easy slicing
taxi = taxi.drop(['pickup_datetime', 'dropoff_datetime', 'trip_duration'], axis=1)
taxi['trip_duration'] = temp

In [9]:
taxi.tail()

,id,vendor_id,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,dist,pick_date,pick_time,drop_time,holiday,holiday_Fri,weekday,trip_duration
1458639,2376096,2,4,-73.982201,40.745522,-73.994911,40.740170,0,1.224311,98,48663,49441,0,1,4,778
1458640,1049543,1,1,-74.000946,40.747379,-73.970184,40.796547,0,6.046037,9,27314,27969,1,1,6,655
1458641,2304944,2,1,-73.959129,40.768799,-74.004433,40.707371,0,7.819693,112,25060,25824,0,1,4,764
1458642,2714485,1,1,-73.982079,40.749062,-73.974632,40.757107,0,1.091878,4,57385,57758,0,0,1,373
1458643,1209952,1,1,-73.979538,40.781750,-73.972809,40.790585,0,1.133330,95,53064,53262,0,0,1,198


In [10]:
taxi.to_csv("edited_taxi.csv", index = False)